# Simple Information Analysis of Chicken Wings

In [ ]:
using Distributed
addprocs(4);

In [ ]:
@everywhere begin
    using Pkg
    Pkg.activate("..")
    using Distributed, Base.Iterators    
    include("../src/binning.jl")
    include("../src/load.jl")
    include("../src/info.jl")
end

In [ ]:
using DelimitedFiles, Plots
include("../src/analysis.jl")

In [ ]:
fs = frames("../videos/Before2-MPGC.mov")[57:456, 61:460, :];

In [ ]:
g = green(fs);

In [ ]:
greengrid = rescale(gridmean(g, 10, 1));

In [ ]:
if haskey(ENV, "GKS_WSTYPE")
    delete!(ENV, "GKS_WSTYPE")
end
p = plot(xlabel="time", ylabel="channel value", legend=false)
for i in 1:size(greengrid, 1), j in 1:size(greengrid, 2)
    plot!(p, greengrid[i,j,:])
end
p

In [ ]:
@time greenmi = analyze(greengrid; nperms=1000);

In [ ]:
anim = @animate for (l, mi) in enumerate(greenmi)
    heatmap(linearize(mi), title="Lag $(l-1)")
end
gif(anim; fps=0.5)

In [ ]:
writedlm("info.txt", hcat(linearize.(greenmi)...))